# Implementando uma MLP básica com Grid Search para escolher melhor os parâmetros no sistema de Mackey-Glass

## 1. Importando as bibliotecas necessárias

### 1.1 Bibliotecas gerais

In [1]:
%matplotlib widget
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns # a biblioteca 'seaborn' contém vários estilos para os gráficos do 'matpĺotlib'

# agora, melhoramos a qualidade de saida e de visualizacao da imagem 
# alem de mudar a fonte padrao para uma do latex
sns.set_style("ticks")
plt.rcParams['savefig.dpi'] = 200
plt.rcParams["figure.dpi"] = 150

plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
plt.style.use('dark_background')

### 1.2 Bibliotecas para MLP

In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')

from tensorflow import keras

In [3]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [4]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor

### 1.3 Bibliotecas dos sistemas caóticos

In [5]:
import sys 
sys.path.insert(0, '../../../../scripts')

import timeseries as times
import mackeyglassequations as mgeq

## 2. Gerando a série temporal

In [6]:
t_inicial = 0
t_final = 5000
tau = 22
n = 10
gamma = 0.1
beta = 0.2
theta = 1

In [7]:
macglass = mgeq.MackeyGlass(tau=tau, gamma=gamma, beta=beta, n=n, theta=theta)

In [8]:
solucoes, instantes_temporais = macglass.calcular(t_inicial = t_inicial, t_final = t_final)

Generating, compiling, and loading C code.
Using default integration parameters.


### Série Temporal de Mackey-Glass

In [9]:
fig, ax = plt.subplots()
ax.set_title('Série temporal de 0 a 600 dias da equação de Mackey-Glass para\n' + r'$\tau =$ ' + str(tau) + r', $\beta =$ ' + str(beta) + r', $\gamma =$ ' + str(gamma) + r', $n =$ ' + str(n) + r' e $\theta =$ ' + str(theta) + ' utilizando $P(0) =$ ' + str(0.1*theta))
ax.plot(instantes_temporais, solucoes, color='Gold')

ax.set_ylabel('$P(t)$')
ax.set_xlabel('$t$')
ax.set_xlim(0,600)
    
ax.grid(True)

fig.tight_layout()
sns.despine()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

### 2.1 Dividindo em um conjunto de treinamento e de teste, para K = 4 e L = 3

In [10]:
K = 4
L = 3
tam_teste = 0.15

In [11]:
x = np.array(solucoes)
x = np.reshape(x, (1, len(x)))
x = x[0]

In [12]:
serie_temporal = times.SerieTemporal(x, K=K, L=L)

In [13]:
serie_temporal.criar_matrizes()

In [14]:
X_treino, X_teste, y_treino, y_teste = serie_temporal.dividir_treino_teste(tam_teste)

## 3. Definindo o modelo para a MLP

### 3.1 Definindo função para criar a MLP

In [15]:
def criar_modelo(batch_normalization='OFF', learning_rate=0.001, activation='selu', init_mode='lecun_normal', n_neurons=30, name="MLP-1-camada-intermediaria"):

    optimizer_gs = optimizer
    optimizer_gs.learning_rate.assign(learning_rate)
    
    model = keras.Sequential(name=name)
    model.add(keras.layers.Dense(K, input_dim=K, name="camada_de_entrada", activation = 'linear'))
    if (batch_normalization == 'ON'):
        model.add(keras.layers.BatchNormalization(name="camada_de_batch_normalization"))
    model.add(keras.layers.Dense(n_neurons, input_dim=K, activation=activation, kernel_initializer=init_mode, name="camada_intermediaria"))
    model.add(keras.layers.Dense(1, activation='linear', name="camada_de_saida"))
    
    model.compile(
        optimizer = optimizer_gs,
        loss = loss)
    
    model.build()
    return model

Utilizaremos os seguintes parâmetros no *Grid Search*:

In [16]:
param_grid = dict(batch_size=[2, 4, 8, 16, 32], 
                  batch_normalization=['ON', 'OFF'], 
                  activation=['selu', 'relu', 'elu', 'sigmoid', 'tanh'], 
                  init_mode = ['lecun_uniform', 'lecun_normal', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform'],
                  n_neurons = [5, 10, 15, 20, 30, 50, 75, 100],
                  learning_rate = [0.001, 0.003, 0.005, 0.008, 0.01])

Para facilitar, dividiremos esse processo em etapas.

### 3.2 Definindo parâmetros que não serão definidos pelo *Grid Search*

In [17]:
loss = "mean_squared_error"
optimizer = keras.optimizers.Nadam()

### 3.3 Definindo e executando o primeiro *Grid Search*

Primeiro, avaliaremos o impacto do *batch size* e da camada de *batch normalization*.

In [71]:
param_grid_1 = dict(batch_size=[2, 4, 8, 16, 32], 
                  batch_normalization=['ON', 'OFF'])

In [72]:
model_cv_1 = KerasRegressor(build_fn=criar_modelo, epochs=100, verbose=0)

In [73]:
grid = GridSearchCV(estimator=model_cv_1, param_grid=param_grid_1, n_jobs=1, cv=4, scoring='neg_mean_squared_error', verbose=1)

In [74]:
grid_result = grid.fit(X_treino, y_treino)

Fitting 4 folds for each of 10 candidates, totalling 40 fits


In [75]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.000548 using {'batch_normalization': 'OFF', 'batch_size': 2}


In [76]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.079693 (0.039688) with: {'batch_normalization': 'ON', 'batch_size': 2}
-0.014648 (0.005031) with: {'batch_normalization': 'ON', 'batch_size': 4}
-0.007042 (0.004442) with: {'batch_normalization': 'ON', 'batch_size': 8}
-0.019222 (0.029663) with: {'batch_normalization': 'ON', 'batch_size': 16}
-0.002713 (0.001854) with: {'batch_normalization': 'ON', 'batch_size': 32}
-0.000548 (0.000218) with: {'batch_normalization': 'OFF', 'batch_size': 2}
-0.000560 (0.000132) with: {'batch_normalization': 'OFF', 'batch_size': 4}
-0.000661 (0.000213) with: {'batch_normalization': 'OFF', 'batch_size': 8}
-0.000614 (0.000241) with: {'batch_normalization': 'OFF', 'batch_size': 16}
-0.001450 (0.001221) with: {'batch_normalization': 'OFF', 'batch_size': 32}


### 3.4 Definindo e executando o segundo *Grid Search*

Agora, avaliaremos o impacto do *learning rate* do otimizador.

In [77]:
model_cv_2 = KerasRegressor(build_fn=criar_modelo, epochs=100, verbose=0, batch_size=4, batch_normalization='OFF')

In [78]:
param_grid_2 = dict(learning_rate=[0.001, 0.003, 0.005, 0.008, 0.01])

In [79]:
grid = GridSearchCV(estimator=model_cv_2, param_grid=param_grid_2, n_jobs=1, cv=4, scoring='neg_mean_squared_error', verbose=1)

In [80]:
grid_result = grid.fit(X_treino, y_treino)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


In [81]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.000467 using {'learning_rate': 0.001}


In [82]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.000467 (0.000137) with: {'learning_rate': 0.001}
-0.000634 (0.000213) with: {'learning_rate': 0.003}
-0.001303 (0.000945) with: {'learning_rate': 0.005}
-0.001094 (0.000368) with: {'learning_rate': 0.008}
-0.000923 (0.000415) with: {'learning_rate': 0.01}


### 3.5 Definindo e executando o terceiro *Grid Search*

Agora, avaliaremos o impacto da função de ativação da camada intermediária.

In [83]:
model_cv_3 = KerasRegressor(build_fn=criar_modelo, epochs=100, verbose=0, batch_size=4, batch_normalization='OFF', learning_rate=0.001)

In [84]:
param_grid_3 = dict(activation=['selu', 'relu', 'elu', 'sigmoid', 'tanh'])

In [85]:
grid = GridSearchCV(estimator=model_cv_3, param_grid=param_grid_3, n_jobs=1, cv=4, scoring='neg_mean_squared_error', verbose=1)

In [86]:
grid_result = grid.fit(X_treino, y_treino)

Fitting 4 folds for each of 5 candidates, totalling 20 fits


In [87]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.000512 using {'activation': 'tanh'}


In [88]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.000671 (0.000351) with: {'activation': 'selu'}
-0.000708 (0.000111) with: {'activation': 'relu'}
-0.000572 (0.000145) with: {'activation': 'elu'}
-0.000926 (0.000323) with: {'activation': 'sigmoid'}
-0.000512 (0.000042) with: {'activation': 'tanh'}


### 3.6 Definindo e executando o quarto *Grid Search*

Agora, avaliaremos o impacto do inicializador da camada intermediária.

In [18]:
model_cv_4 = KerasRegressor(build_fn=criar_modelo, epochs=100, verbose=0, batch_size=4, batch_normalization='OFF', learning_rate=0.001, activation='tanh')

In [19]:
param_grid_4 = dict(init_mode = ['glorot_uniform', 'glorot_normal'])

In [20]:
grid = GridSearchCV(estimator=model_cv_4, param_grid=param_grid_4, n_jobs=1, cv=4, scoring='neg_mean_squared_error', verbose=1)

In [21]:
grid_result = grid.fit(X_treino, y_treino)

Fitting 4 folds for each of 2 candidates, totalling 8 fits


In [22]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.000654 using {'init_mode': 'glorot_normal'}


In [23]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.000667 (0.000392) with: {'init_mode': 'glorot_uniform'}
-0.000654 (0.000139) with: {'init_mode': 'glorot_normal'}


### 3.7 Definindo e executando o quinto *Grid Search*

Agora, avaliaremos o número de neurônios na camada intermediária.

In [24]:
model_cv_5 = KerasRegressor(build_fn=criar_modelo, epochs=100, verbose=0, batch_size=4, batch_normalization='OFF', learning_rate=0.001, activation='tanh', init_mode='glorot_normal')

In [25]:
param_grid_5 = dict(n_neurons = [5, 10, 15, 20, 30, 50, 75, 100])

In [26]:
grid = GridSearchCV(estimator=model_cv_5, param_grid=param_grid_5, n_jobs=1, cv=4, scoring='neg_mean_squared_error')

In [27]:
grid_result = grid.fit(X_treino, y_treino)

In [28]:
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: -0.000501 using {'n_neurons': 5}


In [29]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

-0.000501 (0.000037) with: {'n_neurons': 5}
-0.000663 (0.000188) with: {'n_neurons': 10}
-0.000645 (0.000165) with: {'n_neurons': 15}
-0.000725 (0.000328) with: {'n_neurons': 20}
-0.000575 (0.000145) with: {'n_neurons': 30}
-0.000797 (0.000286) with: {'n_neurons': 50}
-0.000548 (0.000190) with: {'n_neurons': 75}
-0.000585 (0.000092) with: {'n_neurons': 100}


### Treino com o melhor modelo

In [19]:
model = criar_modelo(name='MLP-basica-mackeyglass', batch_normalization='OFF', learning_rate=0.001, activation='tanh', init_mode='glorot_normal', n_neurons=5)

In [20]:
model.summary()

Model: "MLP-basica-mackeyglass"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
camada_de_entrada (Dense)    (None, 4)                 20        
_________________________________________________________________
camada_intermediaria (Dense) (None, 5)                 25        
_________________________________________________________________
camada_de_saida (Dense)      (None, 1)                 6         
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


In [21]:
X_treino, X_val, y_treino, y_val = train_test_split(X_treino, y_treino, test_size=0.1)

In [22]:
early_stopping = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True, monitor='val_loss')

In [23]:
batch_size = 4

In [24]:
history = model.fit(X_treino, y_treino, epochs=100,
                            callbacks=early_stopping, validation_data=(X_val, y_val),
                            batch_size=batch_size)
treinamento = pd.DataFrame(history.history)

Epoch 1/100
955/955 [==============================] - 2s 1ms/step - loss: 0.0314 - val_loss: 0.0046
Epoch 2/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0037 - val_loss: 0.0021
Epoch 3/100
955/955 [==============================] - 2s 2ms/step - loss: 0.0022 - val_loss: 0.0019
Epoch 4/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0021 - val_loss: 0.0019
Epoch 5/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0020 - val_loss: 0.0027
Epoch 6/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0021 - val_loss: 0.0023
Epoch 7/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0020 - val_loss: 0.0018
Epoch 8/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0020 - val_loss: 0.0018
Epoch 9/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0020 - val_loss: 0.0020
Epoch 10/100
955/955 [==============================] - 1s 1ms/step - loss: 0.0020 - val_lo

## Teste com o melhor modelo

In [25]:
y_pred = model.predict(X_teste)

### Gráfico

In [26]:
fig, ax = plt.subplots(figsize=(6.8,6))

ax.plot(instantes_temporais[len(instantes_temporais)-int(len(instantes_temporais)*tam_teste):,], y_teste, color='Gold', label='Valor real')
ax.plot(instantes_temporais[len(instantes_temporais)-int(len(instantes_temporais)*tam_teste):,], y_pred, color='Silver', label='MLP')

ax.set_title('Comparação da predição da MLP com o valor real da equação de Mackey-Glass\n utilizando a rede neural ótima no conjunto de teste')
ax.set_ylabel('$P(t)$')
ax.set_xlabel('$t$')
ax.set_xlim(5000*(1-tam_teste), 5000)
    
ax.grid(True)
sns.despine()
ax.legend()

plt.show()
fig.savefig("../../../../images/mlp-basica/performance/mlp-basica-vs-mackeyglass.png")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

findfont: Font family ['serif'] not found. Falling back to DejaVu Sans.


### Erro Quadrático Médio

In [27]:
from sklearn.metrics import mean_squared_error

In [28]:
mse = mean_squared_error(y_teste, y_pred)
print("MSE = " + str(mse))

MSE = 0.0004040550838816368


### Salvando o Modelo

In [29]:
model.save("../../../../models/mlp-basica/mlp-basica-mackeyglass.h5", include_optimizer=True)